0. Prepare the dataset for the subsequent modelling.
    1. Download the heart disease dataset from https://www.statlearning.com/s/Heart.csv
    2. Load the dataset and drop all variables except the predictors Age, Sex, ChestPain, RestBP, Chol, and the target variable AHD. Drop all rows containing a NaN value.
    3. Onehot encode the variable ChestPain. This means that where you before had a single column with one of four values ['typical', 'asymptomatic', 'nonanginal', 'nontypical'], you will now have four binary columns (their names don't matter), akin to 'ChestPain_typical' 'ChestPain_asymptomatic', 'ChestPain_nonanginal', 'ChestPain_nontypical'. A row that before had ChestPain='typical' will now have ChestPain_typical=1 and the other three columns set to 0, ChestPain='asymptomatic' will have ChestPain_asymptomatic=1 and the other three set to 0, etc.
    4. Binary encode the target variable AHD such that 'No'=0 and 'Yes'=1.

In [7]:
import pandas as pd

# 1: Load the dataset
url = "https://www.statlearning.com/s/Heart.csv"
Heart = pd.read_csv(url)

# 2: Keep relevant columns
Heart = Heart[['Age', 'Sex', 'ChestPain', 'RestBP', 'Chol', 'AHD']]

# 3: Drop rows with any missing values
Heart = Heart.dropna()

# 4: One-hot encode 'ChestPain'
Heart = pd.get_dummies(Heart, columns=['ChestPain'])
Heart.iloc[:, -4:] = Heart.iloc[:, -4:].astype(int)

# 5: Binary encode 'AHD': 'No' → 0, 'Yes' → 1
Heart['AHD'] = Heart['AHD'].map({'No': 0, 'Yes': 1})

Heart.head()


/var/folders/sx/vwqvh81108b_mls63cm7cgcw0000gp/T/ipykernel_87181/515935684.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      0
1      1
2      1
3      0
4      0
      ..
298    0
299    1
300    1
301    0
302    0
Name: ChestPain_asymptomatic, Length: 303, dtype: int64' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  Heart.iloc[:, -4:] = Heart.iloc[:, -4:].astype(int)
/var/folders/sx/vwqvh81108b_mls63cm7cgcw0000gp/T/ipykernel_87181/515935684.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      0
1      0
2      0
3      1
4      0
      ..
298    0
299    0
300    0
301    0
302    1
Name: ChestPain_nonanginal, Length: 303, dtype: int64' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  Heart.iloc[:, -4:] = Heart.iloc[:, -4:].astype(int)
/var/folde

,Age,Sex,RestBP,Chol,AHD,ChestPain_asymptomatic,ChestPain_nonanginal,ChestPain_nontypical,ChestPain_typical
0,63,1,145,233,0,0,0,0,1
1,67,1,160,286,1,1,0,0,0
2,67,1,120,229,1,1,0,0,0
3,37,1,130,250,0,0,1,0,0
4,41,0,130,204,0,0,0,1,0


1. Fit a model using a standard train/validation split through multiple steps.
Through the steps you will practice chaining functions, and you will also create the infrastructure necessary for the remaining tasks.
1.1 Write a function "stratified_split" that takes three arguments: A dataframe, a number of folds, and a list of variables to stratify by. The function should return a list of dataframes, one for each fold, where the dataframes are stratified by the variables in the list. Test that the function works by splitting the dataset into two folds based on 'AHD', 'Age' and 'RestBP' and print the size of each fold, the counts of 0s and 1s in AHD, and the mean of each of 'Age' and 'RestBP' (all these should be printed individually per fold). Ensure that the function does not modify the original dataframe.

In [9]:
# 1: write a function
from sklearn.model_selection import StratifiedKFold
import pandas as pd

def stratified_split(df, n_folds, stratify_vars):
    df_copy = df.copy()
    
    # Create a new column that represents the stratification group
    df_copy['stratify_group'] = df_copy[stratify_vars].astype(str).agg('-'.join, axis=1)
    
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    folds = []
    
    for _, test_idx in skf.split(df_copy, df_copy['stratify_group']):
        fold_df = df_copy.iloc[test_idx].drop(columns=['stratify_group']).copy()
        folds.append(fold_df)
    
    return folds

# 2: test the function 
# Assume 'Heart' DataFrame is already loaded and preprocessed as per your earlier steps

# Apply stratified_split on 'AHD', 'Age', and 'RestBP'
folds = stratified_split(Heart, 2, ['AHD', 'Age', 'RestBP'])

# Print fold statistics
for i, fold in enumerate(folds):
    print(f"Fold {i+1}")
    print(f"Size: {len(fold)}")
    print(f"AHD=0 count: {(fold['AHD'] == 0).sum()}")
    print(f"AHD=1 count: {(fold['AHD'] == 1).sum()}")
    print(f"Mean Age: {fold['Age'].mean():.2f}")
    print(f"Mean RestBP: {fold['RestBP'].mean():.2f}")
    print("-" * 30)




Fold 1
Size: 152
AHD=0 count: 84
AHD=1 count: 68
Mean Age: 54.66
Mean RestBP: 131.25
------------------------------
Fold 2
Size: 151
AHD=0 count: 80
AHD=1 count: 71
Mean Age: 54.22
Mean RestBP: 132.13
------------------------------


/Users/qiqi/miniforge3/envs/python-for-scicomp/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


1.2 Write a function 'fit_and_predict' that takes 4 arguments: A training set, a validation set, a list of predictors, and a target variable. The function should fit a logistic regression model to the training set using the predictors and target variable, and return the predictions of the model on the validation set.

In [12]:

from sklearn.linear_model import LogisticRegression

def fit_and_predict(train_df, valid_df, predictors, target):
    # Create and fit the logistic regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(train_df[predictors], train_df[target])
    
    # Predict probabilities on validation set (only the probability of class 1)
    predictions = model.predict_proba(valid_df[predictors])[:, 1]
    
    return predictions

3. Write a function 'fit_and_predict_standardized' that takes 5 arguments: A training set, a validation set, a list of predictors, a target variable, and a list of variables to standardize. Using a loop (or a scaler), the function should z-score standardize the given variables in both the training set and the validation set based on the mean and standard deviation in the training set. Then, the function should call the 'fit_and_predict' function and return its result. Ensure that the function does not modify the original dataframes. Test the function using the train and validation set from above (e.g. the two folds from the split), while standardizing the 'Age', 'RestBP' and 'Chol' variables (as mentioned above, the target should be AHD, and you should also include the remaining predictors: 'Sex' and the ChestPain-variables)

In [13]:
# 1: write a function 
def fit_and_predict_standardized(train_df, valid_df, predictors, target, standardize_vars):
    # Copy data to avoid modifying the original dataframes
    train_df = train_df.copy()
    valid_df = valid_df.copy()
    
    # Standardize variables using z-score (based on training data)
    for var in standardize_vars:
        mean = train_df[var].mean()
        std = train_df[var].std()
        train_df[var] = (train_df[var] - mean) / std
        valid_df[var] = (valid_df[var] - mean) / std  # use training stats
    
    # Fit and predict using logistic regression
    return fit_and_predict(train_df, valid_df, predictors, target)

# 2: test the function 
# Define columns
standardize_vars = ['Age', 'RestBP', 'Chol']
predictors = ['Age', 'Sex', 'RestBP', 'Chol'] + [col for col in Heart.columns if col.startswith('ChestPain_')]
target = 'AHD'

# Use folds[0] for training, folds[1] for validation
predictions = fit_and_predict_standardized(folds[0], folds[1], predictors, target, standardize_vars)

# Preview predictions
print(predictions[:5])


[0.8924791  0.87586785 0.29629782 0.57033586 0.85776799]


4. Write a function 'fit_and_compute_auc' that takes 5 arguments: A training set, a validation set, a list of predictors, a target variable, and a list of variables to standardize. The function should call the 'fit_and_predict_standardized' function to retrieve out-of-sample predictions for the validation set. Based on these and the ground truth labels in the validation set, it should compute and return the AUC. Test the function using the train and test set from above, while standardizing the 'Age', 'RestBP' and 'Chol' variables (and including the remaining predictors). Print the AUC.

2. Perform a cross-validation.
Use the 'stratified_split' function to split the dataset into 10 folds, stratified on variables you find reasonable. For each fold, use the 'fit_and_compute_auc' function to compute the AUC of the model on the held-out validation set. Print the mean and standard deviation of the AUCs across the 10 folds.

OPTIONAL 3. Use the bootstrap to achieve a distribution of out-of-bag AUCs.
For 100 iterations, create a bootstrap sample by sampling with replacement from the full dataset until you have a training set equal in size to 80% of the original data. Use the observations not included in the bootstrap sample as the validation set for that iteration.. Fit models and calculate AUCs for each iteration. Print the mean and standard deviation of the AUCs.

4. Theory
    1. List some benefits of wrapping code in functions rather than copying and pasting it multiple times.
    2. Explain three classification metrics and their benefits and drawbacks.
    3. Write a couple of sentences comparing the three methods (train/validation, cross-validation, bootstrap) above as approaches to quantify model performance. Which one yielded the best results? Which one would you expect to yield the best results? Can you mention some theoretical benefits and drawbacks with each? Even if you didn't do the optional bootstrap exercise you should reflect on this as an approach.
    4. Why do we stratify the dataset before splitting?
    5. What other use cases can you think of for the bootstrap method?